In [ ]:
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt
import matplotlib.animation as ani

from IPython.display import HTML
from random import choice

In [ ]:
# Define a function to map a 1D array to a 2D/3D lattice, only works on lattice of equal-lateral size in Cartesian system
def map(index, size, dimension): 
    if dimension == 1:
        return index;
    if dimension == 2:
        x = index%size;
        y = np.floor(index/size);
       # return vp.vector(x,y,0);
        return [x, y];
    if dimension == 3:
        sub_index = index%(size**2);
        x = sub_index%size;
        y = np.floor(sub_index/size);
        z = np.floor(index/size**2);
       # return vp.vector(x,y,z)
        return [x, y, z];

In [ ]:
# Define a function to map the 3D vector back into the 1D array index
def antimap(array, size, dimension):
    if dimension == 1:
        return array[0];
    if dimension == 2:
        return array[0] + array[1]*size;
    if dimension == 3:
        return array[0] + array[1]*size + array[2]*size*size;

In [ ]:
# Define a function to find the index of the nearest neighbours
def nNbr(index, size, dimension):
    if dimension == 1:
        return [(index+size-1)%size, (index+size+1)%size]; # apply periodic condition
    
    if dimension == 2:
        pos = map(index,size,dimension);
        north = np.mod(np.add(pos,[0,size+1]), size);
        south = np.mod(np.add(pos,[0,size-1]), size);
        east = np.mod(np.add(pos,[size+1,0]), size);
        west = np.mod(np.add(pos,[size-1,0]),size);
        return [antimap(north, size, 2), antimap(south, size, 2), antimap(east, size, 2), antimap(west, size, 2)];
    
    if dimension == 3:
        pos = map(index,size,dimension);
        north = np.mod(np.add(pos,[0,size+1,0]), size);
        south = np.mod(np.add(pos,[0,size-1,0]), size);
        east = np.mod(np.add(pos,[size+1,0,0]), size);
        west = np.mod(np.add(pos,[size-1,0,0]),size);
        up = np.mod(np.add(pos,[0,0,size+1]), size);
        down = np.mod(np.add(pos,[0,0,size-1]), size);
        return [antimap(north, size, 3), antimap(south, size, 3), antimap(east, size, 3), antimap(west, size, 3), antimap(up, size, 3), antimap(down, size, 3)];

In [ ]:
# Set up the system parameters
temp = 100; # in [K] 
L = 50; # Define the lattice size
dims = 2; # Define the dimensionality of the lattice
N = L**dims; # Compute the total number of the sites
kB = 8.617E-5 # Boltzmann constant in [eV/K]
J = -10.0*kB; # Define the coupling strength between spins
beta = 1/(temp*kB);
MaxIteration = 1000; # Define the maximum amount of iteration
r = 0.2; # dot radius for visualization

In [ ]:
# Set up array containers for data manipulation and initial conditions
#lattice = np.empty((N*N,3),int); # Create a container for the lattice sites
lattice = np.empty((N,dims),float); # Create a container for site positions in the lattice
#spin = np.onesN; # Create a container for the spins for all the lattice sites
spinV = np.empty(N,int); # Create a container for the spins for all the lattice sites
spinC = [None]*N; # Create a container to store vpython objects
nIdx = np.empty((N,dims*2), np.intp); # Create a container to store the indices of the nearest neighbours of each site
probs = np.empty(N,np.float); # Create a container to store in each iteration spin-flip probability
for ii in range(N):
    lattice[ii] = np.subtract(map(ii,L,dims),L/2);
    spinV[ii] = 1; # Create a container to store the values of the spins
    spinC[ii] = 'r';
    nIdx[ii] = nNbr(ii, L, dims);

In [ ]:
# Optional: Randomization of the initial condition
for ii in range(int(N/2)): # Randomly choose half of the lattice to flip the spin 
    kk = choice(range(N));
    spinV[kk] = -1;
    spinC[kk] = 'k'; # color all the spin down as black

In [ ]:
# # Plot the initial condition
# # scat = ax.scatter(lattice[:,0],lattice[:,1],lattice[:,2], s = 200, marker = 's', c=spinC, edgecolor = None);
# figure = plt.figure(figsize=(10,10));
# scat = plt.scatter(lattice[:,0],lattice[:,1],  s = 100, c = spinC, marker = 's', edgecolor = None);
# plt.show()

In [ ]:
# Set the initial condition and start the iteration and implement Metropolis algorithm
MaxIteration = 1000; # Start the iteration count
fig = plt.figure(figsize=(10,10)); # Initialize a figure
scat = plt.scatter(lattice[:,0],lattice[:,1], s=100, c = spinC, marker = 's', edgecolor = None);

def init():

#     scat.set_color = spinC;
    scat.set_color = 'r';
    return scat

def update(frame):
    tempSpin = spinV; # Create a temporary container to store updated spins
    ruler = np.random.rand(len(spinV));
    for ii in range(len(spinV)):
        nIdx[ii] = nNbr(ii, L, dims);
        probs[ii] = np.exp(beta*J*sum(spinV[nIdx[ii]])*spinV[ii])/np.exp(-beta*J*sum(spinV[nIdx[ii]])*spinV[ii]);
        tempSpin[ii] = -spinV[ii] if probs[ii] > ruler[ii] else spinV[ii];
        spinC = 'r' if tempSpin[ii] > 0 else 'k';
    spinV = tempSpin;
    scat.set_color = spinC;
    fig.canvas.draw();
    fig.canvas.flush_events();
    iteration = iteration + 1;
    return scat

cycle = ani.FuncAnimation(fig, update, frames = MaxIteration, init_func=init, interval=1000, blit=False, repeat=False)
HTML(cycle.to_html5_video())